**Mergining the data sets:**
- [Census.gov](https://www.census.gov/data/tables/time-series/demo/popest/2020s-total-metro-and-micro-statistical-areas.html#v2024) - Total Population & Growth
- [Realtors.com](https://www.realtor.com/research/data/) - Monthly Housing Inventory & Hotness 
- [Numbeo.com](https://www.numbeo.com/property-investment/rankings_current.jsp) - Property Price & Investment Scores
- [data.census.gov](https://data.census.gov/table/ACSDT5Y2023.B19013?q=B19013:+Median+Household+Income+in+the+Past+12+Months+(in+2023+Inflation-Adjusted+Dollars)&g=010XX00US$3100000) -Median Household Income (2023 Inflation-Adjusted)

In [146]:
import pandas as pd
import sqlite3
from pathlib import Path

# -----------------------------
# File paths (adjust if needed)
# -----------------------------
numbeo_path      = "NUMBEO_Property_Investment.xlsx"
inventory_path   = "RDC_Inventory_Core_Metrics_Metro.csv"
hotness_path     = "RDC_Inventory_Hotness_Metrics_Metro_History.csv"
population1_path = "population.csv"                       # your earlier population CSV
income_path      = "DCG_Median_Household_Income_2023_Adjusted.csv"
pop_xlsx_path    = "Metro_Population.xlsx"               # new population Excel

# -----------------------------
# 1) Load sources
# -----------------------------
numbeo_df    = pd.read_excel(numbeo_path)
inventory_df = pd.read_csv(inventory_path)
hotness_df   = pd.read_csv(hotness_path)
population1_df = pd.read_csv(population1_path)

# Income CSV has 2 header rows before the table
income_df = pd.read_csv(income_path, skiprows=2)
income_df.columns = ["Geographic Area Name", "Median_HHI_2023_adj", "MoE"]
income_df["Median_HHI_2023_adj"] = (
    income_df["Median_HHI_2023_adj"].astype(str).str.replace(",", "", regex=False).astype(int)
)
income_df["cbsa_title"] = (
    income_df["Geographic Area Name"]
      .str.replace(" Metro Area","", regex=False)
      .str.replace(" Micro Area","", regex=False)
      .str.strip()
)

# New population Excel
pop2_df = pd.read_excel(pop_xlsx_path, sheet_name="Sheet1")

# -----------------------------
# 2) Normalize metro titles
# -----------------------------
def strip_area_suffix(s):
    if pd.isna(s): return s
    return (str(s).replace(" Metro Area","").replace(" Micro Area","").strip())

def normalize_pop_cbsa(s):
    if pd.isna(s): return s
    s = str(s).replace(" MSA","").replace(" Micro Area","").replace(" Metro Area","")
    s = s.replace("–","-")  # en dash -> hyphen
    return s.strip()

inventory_df["cbsa_title"] = inventory_df["cbsa_title"].apply(strip_area_suffix)
hotness_df["cbsa_title"]   = hotness_df["cbsa_title"].apply(strip_area_suffix)
population1_df["cbsa_title"] = (
    population1_df["region"]
      .str.replace(" Metro Area","", regex=False)
      .str.replace(" Micro Area","", regex=False)
      .str.strip()
)
pop2_df["cbsa_title"] = pop2_df["Metropolitan statistical area"].apply(normalize_pop_cbsa)

# -----------------------------
# 3) Derive city/state in NUMBEO
# -----------------------------
def to_city_state(s):
    parts = [p.strip() for p in str(s).split(",")]
    return f"{parts[0]}, {parts[1]}" if len(parts) >= 2 else s

numbeo_df["city_state"]   = numbeo_df["City"].apply(to_city_state)
numbeo_df["numbeo_city"]  = numbeo_df["city_state"].str.split(",").str[0].str.strip()
numbeo_df["numbeo_state"] = numbeo_df["city_state"].str.split(",").str[1].str.strip()

# -----------------------------
# 4) Prefix columns by source (keep join keys unprefixed)
# -----------------------------
def prefixed(df, prefix, keep_cols):
    df = df.copy()
    df.columns = [c if c in keep_cols else f"{prefix}__{c}" for c in df.columns]
    return df

# Keys to keep:
# - NUMBEO: city_state / numbeo_city / numbeo_state (so we can join & keep readable)
# - HOT/INV/POP1/INCOME/POP2: cbsa_title (for metro joins), and month_date_yyyymm on hotness/inventory to pick latest
numbeo_keep = {"city_state","numbeo_city","numbeo_state"}
hot_keep    = {"cbsa_title","month_date_yyyymm"}
inv_keep    = {"cbsa_title","month_date_yyyymm"}
pop1_keep   = {"cbsa_title"}  # if you keep this dataset
income_keep = {"cbsa_title"}
pop2_keep   = {"cbsa_title"}

numbeo_pref = prefixed(numbeo_df,    "numbeo",    numbeo_keep | {"City"})
hot_pref    = prefixed(hotness_df,   "hot",       hot_keep)
inv_pref    = prefixed(inventory_df, "inv",       inv_keep)
pop1_pref   = prefixed(population1_df,"pop1",     pop1_keep)
income_pref = prefixed(income_df,    "income",    income_keep)
pop2_pref   = prefixed(pop2_df,      "pop2",      pop2_keep)

# -----------------------------
# 5) SQLite pipeline
# -----------------------------
conn = sqlite3.connect(":memory:")
numbeo_pref.to_sql("numbeo", conn, index=False, if_exists="replace")
hot_pref.to_sql("hotness", conn, index=False, if_exists="replace")
inv_pref.to_sql("inventory", conn, index=False, if_exists="replace")
pop1_pref.to_sql("population1", conn, index=False, if_exists="replace")
income_pref.to_sql("income", conn, index=False, if_exists="replace")
pop2_pref.to_sql("population2", conn, index=False, if_exists="replace")

# Build normalized CBSA list (from any table that has cbsa_title)
conn.executescript("""
DROP TABLE IF EXISTS rdc_norm;
CREATE TABLE rdc_norm AS
SELECT
  cbsa_title,
  LOWER(REPLACE(TRIM(substr(cbsa_title, 1, instr(cbsa_title || ',', ',') - 1)), '-', ' ')) AS cbsa_cities_blob,
  TRIM(substr(cbsa_title, instr(cbsa_title || ',', ',') + 1)) AS cbsa_states_blob
FROM (
  SELECT DISTINCT cbsa_title FROM inventory
  UNION SELECT DISTINCT cbsa_title FROM hotness
  UNION SELECT DISTINCT cbsa_title FROM population1
  UNION SELECT DISTINCT cbsa_title FROM income
  UNION SELECT DISTINCT cbsa_title FROM population2
)
WHERE cbsa_title IS NOT NULL AND cbsa_title <> '';

DROP VIEW IF EXISTS numbeo_norm;
CREATE VIEW numbeo_norm AS
SELECT DISTINCT
  *,
  LOWER(TRIM(substr(city_state, 1, instr(city_state || ',', ',') - 1))) AS city_lc,
  TRIM(substr(city_state, instr(city_state || ',', ',') + 1)) AS state_code
FROM numbeo
WHERE city_state IS NOT NULL AND city_state <> '';
""")

# Numbeo → CBSA match
conn.executescript("""
DROP TABLE IF EXISTS city_to_cbsa;
CREATE TABLE city_to_cbsa AS
SELECT
  n.city_state,
  n.numbeo_city,
  n.numbeo_state,
  r.cbsa_title AS matched_cbsa_title
FROM numbeo_norm n
LEFT JOIN rdc_norm r
  ON instr(r.cbsa_cities_blob, n.city_lc) > 0
 AND instr(r.cbsa_states_blob, n.state_code) > 0;
""")

# Keep only actual matches
conn.execute("DELETE FROM city_to_cbsa WHERE matched_cbsa_title IS NULL;")

# Deduplicate to one CBSA per city_state
conn.executescript("""
CREATE TEMP TABLE city_to_cbsa_dedup AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY city_state ORDER BY matched_cbsa_title) AS rn
  FROM city_to_cbsa
)
WHERE rn = 1;
DROP TABLE city_to_cbsa;
ALTER TABLE city_to_cbsa_dedup RENAME TO city_to_cbsa;
""")

# Latest month per CBSA for hotness & inventory
final_df = pd.read_sql_query("""
WITH latest_h AS (
  SELECT cbsa_title, MAX(month_date_yyyymm) AS max_month
  FROM hotness
  GROUP BY cbsa_title
),
h_latest AS (
  SELECT h.*
  FROM hotness h
  JOIN latest_h lh
    ON h.cbsa_title = lh.cbsa_title
   AND h.month_date_yyyymm = lh.max_month
),
latest_i AS (
  SELECT cbsa_title, MAX(month_date_yyyymm) AS max_month
  FROM inventory
  GROUP BY cbsa_title
),
i_latest AS (
  SELECT i.*
  FROM inventory i
  JOIN latest_i li
    ON i.cbsa_title = li.cbsa_title
   AND i.month_date_yyyymm = li.max_month
)
SELECT
  -- all NUMBEO columns (prefixed)
  n.*,
  -- mapping keys
  m.city_state,
  m.numbeo_city,
  m.numbeo_state,
  m.matched_cbsa_title,
  -- latest Realtor.com hotness & inventory (prefixed)
  h.*,
  i.*,
  -- population1 (your CSV), income, and population2 (Excel)
  p1.*,
  inc.*,
  p2.*
FROM city_to_cbsa m
JOIN numbeo n
  ON n.city_state = m.city_state
LEFT JOIN h_latest h
  ON m.matched_cbsa_title = h.cbsa_title
LEFT JOIN i_latest i
  ON m.matched_cbsa_title = i.cbsa_title
LEFT JOIN population1 p1
  ON m.matched_cbsa_title = p1.cbsa_title
LEFT JOIN income inc
  ON m.matched_cbsa_title = inc.cbsa_title
LEFT JOIN population2 p2
  ON m.matched_cbsa_title = p2.cbsa_title
ORDER BY m.city_state;
""", conn)

conn.close()

# -----------------------------
# 6) Add a clean CBSA code column (coalesce from any source that has it)
# -----------------------------
def coalesce_cbsa_code(df):
    # Try likely column names after prefixing
    candidates = [
        "inv__cbsa_code", "inv__cbsa",           # Inventory
        "hot__cbsa_code", "hot__cbsa",           # Hotness
        "pop1__cbsa_code", "pop1__cbsa",         # population.csv (if present)
        "income__cbsa_code", "income__cbsa",     # Income file (unlikely)
        "pop2__cbsa_code", "pop2__cbsa",         # Excel pop (unlikely)
    ]
    for c in candidates:
        if c in df.columns:
            df["cbsa_code"] = df[c]
            return df
    # If none found, leave blank so we don't crash
    df["cbsa_code"] = pd.NA
    return df

final_df = coalesce_cbsa_code(final_df)

# -----------------------------
# 7) Save result (≈66 rows)
# -----------------------------
out_path = Path("All_Real_Estate_Data_with_CBSA.xlsx")
final_df.to_excel(out_path, index=False)
print(f"Saved → {out_path.resolve()}")
print(final_df.shape)
final_df

Saved → C:\Users\maktr\Documents\The_Information_Lab\All_Real_Estate_Data_with_CBSA.xlsx
(60, 99)


,City,numbeo__Price To Income Ratio,numbeo__Gross Rental Yield City Centre,numbeo__Gross Rental Yield Outside of Centre,numbeo__Price To Rent Ratio City Centre,numbeo__Price To Rent Ratio Outside Of City Centre,numbeo__Mortgage As A Percentage Of Income,numbeo__Affordability Index,city_state,numbeo_city,...,income__Geographic Area Name,income__Median_HHI_2023_adj,income__MoE,cbsa_title,pop2__Metropolitan statistical area,pop2__2024 Population Estimate,pop2__2020 Population Estimate,pop2__Percent Change in Population,cbsa_title,cbsa_code
0,"Albuquerque, NM, United States",3.00,0.1020,10.90,9.80,9.20,0.2690,3.70,"Albuquerque, NM",Albuquerque,...,"Albuquerque, NM Metro Area",67640,"± 2,150","Albuquerque, NM","Albuquerque, NM MSA",926303,916528,0.010665,"Albuquerque, NM",10740.0
1,"Anchorage, AK, United States",2.90,0.1150,10.30,8.70,9.70,0.2610,3.80,"Anchorage, AK",Anchorage,...,"Anchorage, AK Metro Area",94150,"± 4,935","Anchorage, AK","Anchorage, AK MSA",407213,398328,0.022306,"Anchorage, AK",11260.0
2,"Ann Arbor, MI, United States",3.30,0.1150,10.10,8.70,9.90,0.2960,3.40,"Ann Arbor, MI",Ann Arbor,...,"Ann Arbor, MI Metro Area",83754,"± 4,347","Ann Arbor, MI","Ann Arbor, MI MSA",373875,372258,0.004344,"Ann Arbor, MI",11460.0
3,"Atlanta, GA, United States",2.60,0.1310,14.10,7.60,7.10,0.2310,4.30,"Atlanta, GA",Atlanta,...,"Atlanta-Sandy Springs-Roswell, GA Metro Area",86505,± 862,"Atlanta-Sandy Springs-Roswell, GA","Atlanta–Sandy Springs–Roswell, GA MSA",6411149,6104803,0.050181,"Atlanta-Sandy Springs-Roswell, GA",12060.0
4,"Austin, TX, United States",3.90,0.1060,9.80,9.40,10.20,0.3500,2.90,"Austin, TX",Austin,...,"Austin-Round Rock-San Marcos, TX Metro Area",98508,"± 2,382","Austin-Round Rock-San Marcos, TX","Austin–Round Rock–San Marcos, TX MSA",2550637,2283371,0.117049,"Austin-Round Rock-San Marcos, TX",12420.0
5,"Baltimore, MD, United States",2.60,0.1880,14.60,5.30,6.80,0.2340,4.30,"Baltimore, MD",Baltimore,...,"Baltimore-Columbia-Towson, MD Metro Area",94289,"± 1,744","Baltimore-Columbia-Towson, MD","Baltimore–Columbia–Towson, MD MSA",2859024,2844510,0.005102,"Baltimore-Columbia-Towson, MD",12580.0
6,"Boston, MA, United States",6.30,0.0770,8.50,13.00,11.80,0.5570,1.80,"Boston, MA",Boston,...,"Boston-Cambridge-Newton, MA-NH Metro Area",110697,"± 1,909","Boston-Cambridge-Newton, MA-NH","Boston–Cambridge–Newton, MA-NH MSA",5025517,4941632,0.016975,"Boston-Cambridge-Newton, MA-NH",14460.0
7,"Buffalo, NY, United States",3.20,0.1280,10.30,7.80,9.70,0.2890,3.50,"Buffalo, NY",Buffalo,...,"Buffalo-Cheektowaga, NY Metro Area",69861,"± 1,917","Buffalo-Cheektowaga, NY","Buffalo–Cheektowaga, NY MSA",1160172,1166902,-0.005767,"Buffalo-Cheektowaga, NY",15380.0
8,"Charleston, SC, United States",4.00,0.1290,10.90,7.70,9.20,0.3590,2.80,"Charleston, SC",Charleston,...,"Charleston-North Charleston, SC Metro Area",85165,"± 2,757","Charleston-North Charleston, SC","Charleston–North Charleston, SC MSA",869940,799636,0.087920,"Charleston-North Charleston, SC",16700.0
9,"Charlotte, NC, United States",3.80,0.1170,11.90,8.50,8.40,0.3330,3.00,"Charlotte, NC",Charlotte,...,"Charlotte-Concord-Gastonia, NC-SC Metro Area",81262,"± 1,347","Charlotte-Concord-Gastonia, NC-SC","Charlotte–Concord–Gastonia, NC-SC MSA",2883370,2660329,0.083840,"Charlotte-Concord-Gastonia, NC-SC",16740.0


**Choosing the needed columns**



**From Realtor.com**

* **Hotness metrics (rank, score, demand/supply)**

  * `hot__hotness_rank`
  * `hot__hotness_score`
  * `hot__supply_score`
  * `hot__demand_score`

* **Price trends (median, MoM %, YoY %)**

  * `hot__median_listing_price`
  * `hot__median_listing_price_mm`
  * `hot__median_listing_price_yy`

* **Days on market (DOM) trends**

  * `hot__median_days_on_market`
  * `hot__median_days_on_market_mm`
  * `hot__median_days_on_market_yy`

* **Listing counts & pending ratios**

  * `inv__active_listing_count`
  * `inv__new_listing_count`
  * `inv__pending_ratio`

* **Price change shares**

  * `inv__price_increased_share`
  * `inv__price_reduced_share`

---

**From Numbeo**

* `numbeo__Price To Income Ratio`
* `numbeo__Gross Rental Yield City Centre`
* `numbeo__Gross Rental Yield Outside of Centre`
* `numbeo__Price To Rent Ratio City Centre`
* `numbeo__Price To Rent Ratio Outside Of City Centre`
* `numbeo__Mortgage As A Percentage Of Income`
* `numbeo__Affordability Index`

---

**From Others (Newly Added)**

* **Median Household Income**

  * `income__Median_HHI_2023_adj`

* **Population (2020 & 2024 estimates)**

  * `pop2__2020 Population Estimate`
  * `pop2__2024 Population Estimate`

* **Population Growth %**

  * `pop2__Percent Change in Population`


In [147]:
import pandas as pd
import sqlite3
from collections import defaultdict

# --- 0) Make column names unique BEFORE to_sql ---
final_df.columns = final_df.columns.map(str).str.strip()

counts = defaultdict(int)
new_cols = []
for c in final_df.columns:
    counts[c] += 1
    new_cols.append(c if counts[c] == 1 else f"{c}__dup{counts[c]-1}")
final_df.columns = new_cols
# (If you prefer dropping dupes instead: final_df = final_df.loc[:, ~final_df.columns.duplicated()] )

# --- 1) Load into SQLite ---
conn = sqlite3.connect(":memory:")
final_df.to_sql("df", conn, index=False, if_exists="replace")

# --- 2) Use SQLite quoting ("") not [] and avoid bare identifiers starting with digits ---
query = """
SELECT 
  df.cbsa_title AS Metropolitan_Area,
  df.City AS Numbeo_City,
  df.cbsa_code AS cbsa_code,
  df.hot__hotness_rank,
  df.hot__hotness_score,
  df.hot__supply_score,
  df.hot__demand_score,

  df.hot__median_listing_price,
  df.hot__median_listing_price_mm,
  df.hot__median_listing_price_yy,

  df.hot__median_days_on_market,
  df.hot__median_days_on_market_mm,
  df.hot__median_days_on_market_yy,

  df.inv__active_listing_count,
  df.inv__new_listing_count,
  df.inv__pending_ratio,

  df.inv__price_increased_share,
  df.inv__price_reduced_share,

  df."numbeo__Price To Income Ratio" AS numbeo_Price_To_Income_Ratio,
  df."numbeo__Gross Rental Yield City Centre" AS numbeo_GRY_Center,
  df."numbeo__Gross Rental Yield Outside of Centre" AS numbeo_GRY_Outside,
  df."numbeo__Price To Rent Ratio City Centre" AS numbeo_PTR_Center,
  df."numbeo__Price To Rent Ratio Outside Of City Centre" AS numbeo_PTR_Outside,
  df."numbeo__Mortgage As A Percentage Of Income" AS numbeo_MortgagePctIncome,
  df."numbeo__Affordability Index" AS numbeo_AffordabilityIndex,

  df.[income__Median_HHI_2023_adj] AS Median_HHI_2023_adj,

  df."pop2__2020 Population Estimate" AS pop_2020_est,
  df."pop2__2024 Population Estimate" AS pop_2024_est,
  df."pop2__Percent Change in Population" AS pop_pct_change
FROM df
"""

df = pd.read_sql_query(query, conn)
conn.close()

# --- 3) Safe handling of 'Region' and numeric filtering ---
cols_to_convert = df.columns
if 'Region' in cols_to_convert:
    cols_to_convert = cols_to_convert.drop('Region')

# Optional: keep only rows where all selected columns are non-zero
# (coerce non-numeric to NaN to avoid comparison warnings)
num_like = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

df



,Metropolitan_Area,Numbeo_City,cbsa_code,hot__hotness_rank,hot__hotness_score,hot__supply_score,hot__demand_score,hot__median_listing_price,hot__median_listing_price_mm,hot__median_listing_price_yy,...,numbeo_GRY_Center,numbeo_GRY_Outside,numbeo_PTR_Center,numbeo_PTR_Outside,numbeo_MortgagePctIncome,numbeo_AffordabilityIndex,Median_HHI_2023_adj,pop_2020_est,pop_2024_est,pop_pct_change
0,"Albuquerque, NM","Albuquerque, NM, United States",10740.0,208,33.946488,25.752508,42.140468,420000.00,-0.003500,0.025015,...,0.1020,10.90,9.80,9.20,0.2690,3.70,67640,916528,926303,0.010665
1,"Anchorage, AK","Anchorage, AK, United States",11260.0,178,40.802676,64.548495,17.056856,475000.00,-0.015544,0.034351,...,0.1150,10.30,8.70,9.70,0.2610,3.80,94150,398328,407213,0.022306
2,"Ann Arbor, MI","Ann Arbor, MI, United States",11460.0,56,77.424749,82.943144,71.906355,517500.00,-0.029991,-0.017094,...,0.1150,10.10,8.70,9.90,0.2960,3.40,83754,372258,373875,0.004344
3,"Atlanta-Sandy Springs-Roswell, GA","Atlanta, GA, United States",12060.0,235,25.919732,40.468227,11.371237,419945.00,-0.002506,-0.011778,...,0.1310,14.10,7.60,7.10,0.2310,4.30,86505,6104803,6411149,0.050181
4,"Austin-Round Rock-San Marcos, TX","Austin, TX, United States",12420.0,283,11.371237,19.063545,3.678930,510950.00,-0.026669,-0.048510,...,0.1060,9.80,9.40,10.20,0.3500,2.90,98508,2283371,2550637,0.117049
5,"Baltimore-Columbia-Towson, MD","Baltimore, MD, United States",12580.0,122,56.354515,89.632107,23.076923,399900.00,-0.000248,0.066400,...,0.1880,14.60,5.30,6.80,0.2340,4.30,94289,2844510,2859024,0.005102
6,"Boston-Cambridge-Newton, MA-NH","Boston, MA, United States",14460.0,45,80.769231,82.943144,78.595318,841950.00,-0.015233,-0.014052,...,0.0770,8.50,13.00,11.80,0.5570,1.80,110697,4941632,5025517,0.016975
7,"Buffalo-Cheektowaga, NY","Buffalo, NY, United States",15380.0,31,86.287625,96.655518,75.919732,299450.00,-0.001501,0.069655,...,0.1280,10.30,7.80,9.70,0.2890,3.50,69861,1166902,1160172,-0.005767
8,"Charleston-North Charleston, SC","Charleston, SC, United States",16700.0,150,47.826087,45.150502,50.501672,518750.00,-0.030374,-0.002394,...,0.1290,10.90,7.70,9.20,0.3590,2.80,85165,799636,869940,0.087920
9,"Charlotte-Concord-Gastonia, NC-SC","Charlotte, NC, United States",16740.0,162,44.481605,43.812709,45.150502,449432.50,-0.011149,0.023764,...,0.1170,11.90,8.50,8.40,0.3330,3.00,81262,2660329,2883370,0.083840


In [148]:
import numpy as np
from scipy.stats import zscore

# Define the normalization helper
def Zscore_Normalize(series):
    # Get z-scores
    z_scores = zscore(series, nan_policy='omit')
    # Min-max scale z-scores to 0–1, then return
    return (z_scores - np.nanmin(z_scores)) / (np.nanmax(z_scores) - np.nanmin(z_scores))


1. **Liquidity Score**

**What it is:** How quickly homes are selling in a city.

**Why it matters:** Faster sales mean money moves quicker and properties are easier to buy or sell without sitting on the market.

**Goal:** Higher = faster market turnover & strong buyer activity.

**Inputs:**

- ``inv__pending_ratio`` (higher = better)
- ``hot__median_days_on_market`` (lower = better → inverse)
- ``inv__active_listing_count`` (higher = better for liquidity, but high supply can sometimes slow absorption)



In [149]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

df["Liquidity_Score"] = (
    Zscore_Normalize(df["inv__pending_ratio"]) +
    Zscore_Normalize(df["hot__median_days_on_market"].max() - df["hot__median_days_on_market"]) +
    Zscore_Normalize(df["inv__active_listing_count"])
) / 3 * 100

2. **Housing Market Growth Score**

**What it is:** How much home prices and market activity are increasing.

**Why it matters:** Price growth can increase the value of an investment over time, giving higher resale profits.

**Goal:** Higher = faster appreciation potential

**Inputs:**
- ``hot__hotness_rank`` (lower rank = better → inverse)
- ``hot__median_listing_price_mm`` (monthly % change)
- ``hot__median_listing_price_yy`` (yearly % change)



In [150]:
df["Housing_Market_Growth_Score"] = (
    Zscore_Normalize(df["hot__hotness_rank"].max() - df["hot__hotness_rank"]) +
    Zscore_Normalize(df["hot__median_listing_price_mm"]) +
    Zscore_Normalize(df["hot__median_listing_price_yy"])
) / 3 * 100


3. **Yield Score**

**What it is:** How much rental income you get compared to the cost of buying.

**Why it matters:** Higher yields mean stronger cash flow, which improves overall return on investment.

**Goal:** Higher = better rental return relative to property price.

**Inputs:**
- ``numbeo_GRY_Center`` (Gross Rental Yield City Centre)
- ``numbeo_GRY_Outside`` (Gross Rental Yield Outside of Centre)
- ``numbeo_PTR_Center`` (Price To Rent Ratio City Centre → lower = better → inverse)
- ``numbeo_PTR_Outside`` (Price To Rent Ratio Outside of Centre → lower = better → inverse)



In [151]:
df["Yield_Score"] = (
    Zscore_Normalize(df["numbeo_GRY_Center"]) +
    Zscore_Normalize(df["numbeo_GRY_Outside"]) +
    Zscore_Normalize(df["numbeo_PTR_Center"].max() - df["numbeo_PTR_Center"]) +
    Zscore_Normalize(df["numbeo_PTR_Outside"].max() - df["numbeo_PTR_Outside"])
) / 4 * 100


4. **Affordability Score**

**What it is:** How easy it is for the average household to buy a home.

**Why it matters:** Affordable markets tend to attract more buyers and renters, helping maintain steady demand.

**Goal:** Higher = more affordable and financially sustainable.

**Inputs:**
- ``numbeo_Price_To_Income_Ratio`` (lower = better → inverse)
- ``numbeo_MortgagePctIncome`` (lower = better → inverse)
- ``numbeo_AffordabilityIndex`` (higher = better)




In [152]:
df["Affordability_Score"] = (
    Zscore_Normalize(df["numbeo_Price_To_Income_Ratio"].max() - df["numbeo_Price_To_Income_Ratio"]) +
    Zscore_Normalize(df["numbeo_MortgagePctIncome"].max() - df["numbeo_MortgagePctIncome"]) +
    Zscore_Normalize(df["numbeo_AffordabilityIndex"])
) / 3 * 100


5. **Income Strength**

**What it is:** How much money the average household earns each year.

**Why it matters:** Higher incomes support higher rents and make tenant payment risk lower.

**Goal:** Higher = stronger tenant/buyer purchasing power.

**Inputs:**

- ``Median_HHI_2023_adj`` (Median Household Income, inflation-adjusted)



In [153]:
df["Income_Strength"] = Zscore_Normalize(df["Median_HHI_2023_adj"]) * 100

6. **Population Momentum**

**What it is:** How fast the population is growing in the area.

**Why it matters:** A growing population creates future housing demand, which can push up prices and rents.

**Goal**: Higher = faster demographic growth, more housing demand.

**Inputs:**
- ``pop_pct_change`` (% change from 2020 to 2024)



In [154]:
df["Population_Momentum"] = Zscore_Normalize(df["pop_pct_change"]) * 100

**Composite Opportunity Score**

**Goal:** Create a single ranking metric that blends all key investment dimensions — liquidity, growth, yield, affordability, income strength, and population momentum — into one 0–100 score.

**Inputs:**
- ``Liquidity_Score``
- ``Housing_Market_Growth_Score``
- ``Yield_Score``
- ``Affordability_Score``
- ``Income_Strength``
- ``Population_Momentum``

In [155]:
df["Composite_Opportunity_Score"] = (
    df["Liquidity_Score"] * 0.20 +
    df["Housing_Market_Growth_Score"] * 0.20 +
    df["Yield_Score"] * 0.20 +
    df["Affordability_Score"] * 0.15 +
    Zscore_Normalize(df["Income_Strength"]) * 0.15 +
    Zscore_Normalize(df["Population_Momentum"]) * 0.10
) * 100


In [156]:
df

,Metropolitan_Area,Numbeo_City,cbsa_code,hot__hotness_rank,hot__hotness_score,hot__supply_score,hot__demand_score,hot__median_listing_price,hot__median_listing_price_mm,hot__median_listing_price_yy,...,pop_2020_est,pop_2024_est,pop_pct_change,Liquidity_Score,Housing_Market_Growth_Score,Yield_Score,Affordability_Score,Income_Strength,Population_Momentum,Composite_Opportunity_Score
0,"Albuquerque, NM","Albuquerque, NM, United States",10740.0,208,33.946488,25.752508,42.140468,420000.00,-0.003500,0.025015,...,916528,926303,0.010665,23.959198,48.287949,48.481046,73.709405,7.201579,32.582156,3524.543389
1,"Anchorage, AK","Anchorage, AK, United States",11260.0,178,40.802676,64.548495,17.056856,475000.00,-0.015544,0.034351,...,398328,407213,0.022306,36.818167,44.666705,50.142888,75.097672,35.953667,39.959012,3768.409224
2,"Ann Arbor, MI","Ann Arbor, MI, United States",11460.0,56,77.424749,82.943144,71.906355,517500.00,-0.029991,-0.017094,...,372258,373875,0.004344,38.227565,38.817193,49.483967,69.413197,24.678423,28.576078,3578.331833
3,"Atlanta-Sandy Springs-Roswell, GA","Atlanta, GA, United States",12060.0,235,25.919732,40.468227,11.371237,419945.00,-0.002506,-0.011778,...,6104803,6411149,0.050181,41.714010,39.186804,64.457628,80.807337,27.662090,57.624323,4129.190642
4,"Austin-Round Rock-San Marcos, TX","Austin, TX, United States",12420.0,283,11.371237,19.063545,3.678930,510950.00,-0.026669,-0.048510,...,2283371,2550637,0.117049,23.405437,9.326556,46.196310,61.461808,40.680246,100.000000,2516.595226
5,"Baltimore-Columbia-Towson, MD","Baltimore, MD, United States",12580.0,122,56.354515,89.632107,23.076923,399900.00,-0.000248,0.066400,...,2844510,2859024,0.005102,42.341781,68.128397,76.815769,80.675931,36.104423,29.056884,4964.179260
6,"Boston-Cambridge-Newton, MA-NH","Boston, MA, United States",14460.0,45,80.769231,82.943144,78.595318,841950.00,-0.015233,-0.014052,...,4941632,5025517,0.016975,34.513046,51.403792,31.779589,35.819699,53.900132,36.580898,2902.967163
7,"Buffalo-Cheektowaga, NY","Buffalo, NY, United States",15380.0,31,86.287625,96.655518,75.919732,299450.00,-0.001501,0.069655,...,1166902,1160172,-0.005767,59.892772,78.304316,53.281315,70.757663,9.610421,22.168388,4894.591394
8,"Charleston-North Charleston, SC","Charleston, SC, United States",16700.0,150,47.826087,45.150502,50.501672,518750.00,-0.030374,-0.002394,...,799636,869940,0.087920,26.879673,30.377209,55.378871,60.029739,26.208759,81.540340,3165.246486
9,"Charlotte-Concord-Gastonia, NC-SC","Charlotte, NC, United States",16740.0,162,44.481605,43.812709,45.150502,449432.50,-0.011149,0.023764,...,2660329,2883370,0.083840,33.588437,47.786715,55.484611,63.244293,21.975662,78.954510,3697.051444
